In [1]:
import os, subprocess
import json
import uproot3
import awkward as ak
import numpy as np
from coffea import processor, util, hist
import pickle

from plotter import *

In [2]:
lumis = {}
lumis['2016'] = 35.9
lumis['2017'] = 41.5
lumis['2018'] = 59.9

with open('xsec.json') as f:
  xs = json.load(f)

with open('pmap.json') as f:
  pmap = json.load(f)

systematics = ['nominal',
               'jet_triggerUp','jet_triggerDown',
               'btagWeightUp','btagWeightDown','btagEffStatUp','btagEffStatDown',
               'UESUp','UESDown','JESUp','JESDown','JERUp','JERDown',
               'pileup_weightUp','pileup_weightDown'
              ]

In [3]:
deta_cut = 3.5
mjj_cut = 1000

deta_cut_mucr = 0
mjj_cut_mucr = 0

In [4]:
year = '2017'
nfiles = len(subprocess.getoutput("ls infiles-split/"+year+"*.json").split())
outsum = processor.dict_accumulator()

In [5]:
repickle=True

# Check if pickle exists, and don't re-create it if it does
picklename = 'pickles/'+str(year)+'_templates.pkl'
if os.path.isfile(picklename):
    repickle=False

In [6]:
# Load all files - this takes a while
if repickle:
    nfiles = len(subprocess.getoutput("ls infiles-split/"+year+"*.json").split())
    for n in range(1,nfiles+1):

        with open('infiles-split/'+year+'_'+str(n)+'.json') as f:
            infiles = json.load(f)
    
        filename = '/myeosdir/vbf-category/outfiles/'+year+'_'+str(n)+'.coffea'
        if os.path.isfile(filename):
            out = util.load(filename)
            outsum.add(out)
        else:
            print('Missing file '+str(n),infiles.keys())
            #print("File " + filename + " is missing")
        
    scale_lumi = {k: xs[k] * 1000 *lumis[year] / w for k, w in outsum['sumw'].items()}
    outsum['templates-vbf'].scale(scale_lumi, 'dataset')
    templates = outsum['templates-vbf'].group('dataset', hist.Cat('process', 'Process'), pmap)

    outfile = open(picklename, 'wb')
    pickle.dump(templates, outfile, protocol=-1)
    outfile.close()

Missing file 2416 dict_keys(['WJetsToLNu_HT-800To1200_TuneCP5_13TeV-madgraphMLM-pythia8'])


/opt/conda/lib/python3.8/site-packages/coffea/hist/hist_tools.py:376: RuntimeWarning: Not all requested indices present in <Cat (name=dataset) instance at 0x7f9c39c5a700>
  warnings.warn(


In [7]:
# Read the histogram from the pickle file
templates = pickle.load(open(picklename,'rb'))
templates_vbf = templates.integrate('region','signal').integrate('deta',int_range=slice(deta_cut,7)).integrate('mjj',int_range=slice(mjj_cut,4000))
templates_vbf_mucr = templates.integrate('region','muoncontrol').integrate('deta',int_range=slice(deta_cut_mucr,7)).integrate('mjj',int_range=slice(mjj_cut_mucr,4000))

In [8]:
templates_vbf.sum('pt1','msd1').integrate('ddb1',int_range=slice(0,0.89)).integrate('systematic','nominal').values()

{('ZH',): array(0.1281014),
 ('WH',): array(0.30197725),
 ('ttH',): array(0.35629802),
 ('VBF',): array(4.88352433),
 ('ggF',): array(0.93687002),
 ('ggF-powheg',): array(2.97746386),
 ('QCD',): array(171858.32823406),
 ('VV',): array(43.05226924),
 ('Wjets',): array(3479.14796864),
 ('Zjets',): array(1499.54278529),
 ('ttbar',): array(568.41891345),
 ('singlet',): array(115.38995142),
 ('data',): array(178446.),
 ('muondata',): array(4184.)}

In [9]:
templates_vbf.identifiers('systematic')

[<StringBin (JERDown) instance at 0x7f9c7ff6bd60>,
 <StringBin (JERUp) instance at 0x7f9c7ff6bcd0>,
 <StringBin (JESDown) instance at 0x7f9c7ff6bc10>,
 <StringBin (JESUp) instance at 0x7f9c7ff6bbb0>,
 <StringBin (UESDown) instance at 0x7f9c7ff6bc70>,
 <StringBin (UESUp) instance at 0x7f9c7ff6b8e0>,
 <StringBin (W_d2kappa_EWDown) instance at 0x7f9c39c40940>,
 <StringBin (W_d2kappa_EWUp) instance at 0x7f9c9c378340>,
 <StringBin (W_d3kappa_EWDown) instance at 0x7f9c7ff6bdc0>,
 <StringBin (W_d3kappa_EWUp) instance at 0x7f9c7ff6bfa0>,
 <StringBin (Z_d2kappa_EWDown) instance at 0x7f9c39c40100>,
 <StringBin (Z_d2kappa_EWUp) instance at 0x7f9c39c40700>,
 <StringBin (Z_d3kappa_EWDown) instance at 0x7f9c39c405b0>,
 <StringBin (Z_d3kappa_EWUp) instance at 0x7f9c39c40910>,
 <StringBin (btagEffStatDown) instance at 0x7f9c7ff6ba90>,
 <StringBin (btagEffStatUp) instance at 0x7f9c7ff6ba00>,
 <StringBin (btagWeightDown) instance at 0x7f9c7ff6ba60>,
 <StringBin (btagWeightUp) instance at 0x7f9c7ff6bb50>

In [12]:
os.system('rm '+year+'/1-signalregion.root')
fout = uproot3.create(year+'/1-signalregion.root')
for p in pmap.keys():  
    print(p)
    if "data" in p:
        s = "nominal"
        h = templates_vbf.sum('pt1').integrate('systematic',s).integrate('ddb1',int_range=slice(0.89,1)).integrate('process',p)

        fout["pass_"+p+"_"+s] = hist.export1d(h)
        h = templates_vbf.sum('pt1').integrate('systematic',s).integrate('ddb1',int_range=slice(0,0.89)).integrate('process',p)
        fout["fail_"+p+"_"+s] = hist.export1d(h)
    else:
        for s in systematics:
            h = templates_vbf.sum('pt1').integrate('systematic',s).integrate('ddb1',int_range=slice(0.89,1)).integrate('process',p)
            fout["pass_"+p+"_"+s] = hist.export1d(h)
            h = templates_vbf.sum('pt1').integrate('systematic',s).integrate('ddb1',int_range=slice(0,0.89)).integrate('process',p)
            fout["fail_"+p+"_"+s] = hist.export1d(h)

fout.close()

ZH
WH
ttH
VBF
ggF
ggF-powheg
QCD
VV
Wjets
Zjets
ttbar
singlet
data
muondata


In [13]:
ptbins = [450, 550, 1200]
os.system('rm '+year+'/2pt-signalregion.root')
fout = uproot3.create(year+'/2pt-signalregion.root')

for i,b in enumerate(ptbins[:-1]):
    for p in pmap.keys(): 
        print(p)
        if "data" in p:
            s = "nominal"
            h = templates_vbf.integrate('systematic',s).integrate('pt1',int_range=slice(ptbins[i],ptbins[i+1])).integrate('ddb1',int_range=slice(0.89,1)).integrate('process',p)
            fout["pass_pt"+str(i+1)+"_"+p+"_"+s] = hist.export1d(h)
            h = templates_vbf.integrate('systematic',s).integrate('pt1',int_range=slice(ptbins[i],ptbins[i+1])).integrate('ddb1',int_range=slice(0,0.89)).integrate('process',p)
            fout["fail_pt"+str(i+1)+"_"+p+"_"+s] = hist.export1d(h)
        else:
            for s in systematics:
                h = templates_vbf.integrate('systematic',s).integrate('pt1',int_range=slice(ptbins[i],ptbins[i+1])).integrate('ddb1',int_range=slice(0.89,1)).integrate('process',p)
                fout["pass_pt"+str(i+1)+"_"+p+"_"+s] = hist.export1d(h)
                h = templates_vbf.integrate('systematic',s).integrate('pt1',int_range=slice(ptbins[i],ptbins[i+1])).integrate('ddb1',int_range=slice(0,0.89)).integrate('process',p)
                fout["fail_pt"+str(i+1)+"_"+p+"_"+s] = hist.export1d(h)

fout.close()

ZH
WH
ttH
VBF
ggF
ggF-powheg
QCD
VV
Wjets
Zjets
ttbar
singlet
data
muondata
ZH
WH
ttH
VBF
ggF
ggF-powheg
QCD
VV
Wjets
Zjets
ttbar
singlet
data
muondata


In [15]:
os.system('rm '+year+'/muonCR.root')
fout = uproot3.create(year+'/muonCR.root')
for p in pmap.keys():  
    print(p)
    if "data" in p:
        s = "nominal"
        h = templates_vbf_mucr.integrate('systematic',s).sum('pt1').integrate('ddb1',int_range=slice(0.89,1)).integrate('process',p)
        fout["pass_"+p+"_"+s] = hist.export1d(h)
        h = templates_vbf_mucr.integrate('systematic',s).sum('pt1').integrate('ddb1',int_range=slice(0,0.89)).integrate('process',p)
        fout["fail_"+p+"_"+s] = hist.export1d(h)
    else:
        for s in systematics:
            h = templates_vbf_mucr.integrate('systematic',s).sum('pt1').integrate('ddb1',int_range=slice(0.89,1)).integrate('process',p)
            fout["pass_"+p+"_"+s] = hist.export1d(h)
            h = templates_vbf_mucr.integrate('systematic',s).sum('pt1').integrate('ddb1',int_range=slice(0,0.89)).integrate('process',p)
            fout["fail_"+p+"_"+s] = hist.export1d(h)

ZH
WH
ttH
VBF
ggF
ggF-powheg
QCD
VV
Wjets
Zjets
ttbar
singlet
data
muondata
